Application of Newton's methods for solving load flow problems
--
Rene Verinaud Anguita Junior

*Abstract* - Electricity distributors, responsible for the operation and planning of the expansion, need to have real-time knowledge of the point of operation of their networks. For this, they calculate various load flows, in order to always stay updated with the events of the network. There are several types of methods for solving this problem, having the criterion to choose, due to the precision of the result or the computational time. This work presents the implementation of two of these methods, and compares the results found, using an IEEE14 system bars, in order to analyze the result, with the computational cost used. Finding in small systems, minimal differences, but with significant computational gain.

*Keywords* - Reproducibility, Power Flow, Python.

# INTRODUCTION
The calculations of the power flow are indispensable for the planning and operation of the electric power networks, this is due to the importance of having knowledge of the state of the network, the voltage and phase levels. Generally, these networks are represented in a simplified, single-line manner, and calculations are performed in p.u. Another relevant aspect to this analysis is the choice of the method used, with some prioritizing computational time, but not guaranteeing the global optimum (heuristic methods) and others that prioritize the result, but consuming a lot of processing (linear).

The work of [1], presents an approach for calculating the power flow, based on the radial electrical energy distribution system. The problem was formulated to minimize losses and approached the problem in a conical way. The work demonstrates the evolution of the need for interactions, with the increase of the used system, having also an increase in the computational time used to find a solution. [2] also discusses this relation, showing a possible approach to Newton's method, but discussing the variation of the error factor and the number of iterations necessary to find a solution.

The author's [3] did some research and presents a very detailed theoretical explanation of Newton's method. Demonstrating the possible divergence that the problem can assume, by the inversion of the Jacobian matrix. Adapting the method for large systems, where by the inversion of the Jacobian matrix, in addition to the extremely heavy problem, its computational time can suffer a divergence in an iteration, throwing out the whole process until that moment.

In this work will be presented the method of Complete Newton and Fast Decoupled, applied in the resolution of the load flow. Where the theory behind the methodology will be presented, carrying out tests and justifying which one is the fastest. Thus, the objective of this work is to compare these two methods, demonstrating mathematically which one is faster and with the results, justify them, with the tests being carried out in an electric power transmission system.

The work is organized as follows: Chapter 2 there is the explanation and simplifications adopted for Newton Decoupled Fast and Complete Newton. In Chapter 3, the test performed, using a transmission system and discussing the results found. In Chapter 4, you find a conclusion to the work.

# POWER FLOW MODEL

In Figure 1 shows a model of electrical energy distribution. You can see one of the three bars $ k, i, j $, which has a voltage $ V_ {i, d} $. A $ d $ component represents the period these variables are in. Between the bars there is a transmission line, with resistances, impedances and reactances. Above the line, it is the direction of the flow in the line, the active and reactive powers, and its current. In each bar, at the bottom, an injection and demand consumption is displayed, with the direction of the arrow representing these characteristics. The transmission lines have only one differential from this illustration is the shunt capacitors. As they are miles long, it is possible to find a capacity between lines and ground.

![Electricity distribution model with three bars.](../figures/Bar.png)

Source: Retrieved from [4].

To calculate the power flow between the bars $ i, j $, is used the relation of Equations \ref{eq:P_outros_metodos}. Where the first equation $ P_ {i, j} $ represents the direction from $ i $ to $ j $ and $ P_ {j, i} $, the opposite direction. The equations present components correlated to the voltage of each bar $ V $ and of the phase $ \theta $, depending on the direction being calculated. In the equations, the variables $ g $ and $ b $, are the real and imaginary components of the line connected in each bar, respectively.


$\left\{ \begin{array}{lll}
    P_{i,j} = V^2_i g_{i,j} - V_i V_j ( g_{i,j} cos \theta_{i,j} + b_{i,j} sen \theta_{i,j}) \\
    P_{j,i} = V^2_j g_{i,j} - V_i V_j ( g_{i,j} cos \theta_{i,j} - b_{i,j} sen \theta_{i,j}) 
\end{array}\right.$
	 
        
As the Equation mentioned above calculates only the local values, it is necessary to perform the calculation of the system as a whole, to know if there will be a voltage outside of acceptable parameters on any bus, and what the power supplied by the substation should be, to provide the demand of the system. To carry out this analysis, Newton's method was adapted to the charge flow problem, thus originating the Newton Complete methodology. This method calculates the system components, in iterations, finding the mismathes $ \Delta $ (errors) for each one: of power $ \Delta P $ and $ \Delta Q $; voltage $ \Delta V $ and phase $ \Delta \theta $. Thus, in each iteration the value of $ \Delta P $ and $ \Delta Q $ is found, after performing this calculation, it is possible to find $ \Delta \theta $ and $ \Delta V $, which are added in the voltage and phase. In this way, a value closer to the optimum is seen in each iteration. It is necessary to invert the matrices in each iteration, having dimensions of the number of bars x number of bars, and demanding a high computational cost to perform these calculations.

The electric power transmission system has a voltage range, in alternating current, from 138 kV to 765 kV, within this range the voltage classes vary from 230 kV, 345 kV, 440 kV and 500 kV. As the tension levels are very high, in this type of system, it is possible to consider certain simplifications, adopting:

* $\theta_{i,j} \approx 0$, therefore $cos (\theta_{i,j}) \approx 1$.
* $B_{i,i}V^2_{i}>>Q_i$ 
* $V_k \approx 1  pu$
 
    
The Fast Decoupled Newton is based on the previous method, with an implementation of the simplifications mentioned above, with the flowchart of this process that can be seen in Figure \ref{diag:desacopladorapido}. Thus, the method first calculate the physical parameters of the system, and the first mismatch of the active power $ \Delta P $. The next step is the beginning of the iterative process, where a comparison of the highest mismatch value of the active power $ \Delta P $ with the error factor $ \zeta $. If it is bigger, it is calculated the mismatch of phase $ \Delta \theta $, the phase in the following iteration is updated, increasing $ p $, matching $ KQ = 1 $ and calculating the mismatch of the reactive power $ \Delta Q $. Then, if the value of $ \Delta P $ is lower than the value of $ \zeta $, the stopping criterion $ KP $ is equal to zero and the value of $ KQ $ is compared. If it is equal to zero, the problem found a solution, if not, the problem goes to the calculation of $ \Delta Q $.

This process is called a half active iteration. The second part, called reactive half iteration, has the same characteristics as the previous one, but now the $ \Delta Q $ is compared with the $ \zeta $, if it is bigger, the $ \Delta V $ is calculated and the voltage is updated for the next iteration, incrementing the $ q $ iteration counter and making it equal to $ KP $, going back to the beginning of the iterative process and recalculating the $ \Delta P $. If the highest value of $ \Delta Q $ is lower than $ \zeta $, the process equals $ KQ = 0 $, and checks whether the value is $ KP = 0 $, if yes, a solution has been found, if not, the process recalculates the $ \Delta P $.

![](../figures/Flowchart.png)

# TESTS AND RESULTS

The tests used the language solver and Python 3.6.5, being used on a Acer Predator Helios 300 computer, with an i7-7700HQ processor and a frequency of 2.8 GHz. The libraries used were Numpy and Matplotlib, it was also used the Jupyter Notebook program for the documentation. The code and its versions are located in a repository on GitHub [5], where you have the history and an explanation of the code, including the database used. For the tests, the IEEE14 system was used, shown by Figure 3. This system was retrieved from [6], with its data, having 14 bars, with 20 lines. For the test, $ V ^ {base} = 230 kV $, $ S ^ {base} = 100 MVA $, $ \zeta = 10 ^ {- 5} $ were used and the iterations $ p $ and $ q $ were limited to 200.


![IEEE14 system topology.](../figures/14bus600.tif)
	 
As explained in the theory chapter, demand data varies over time, but for the calculation of operation, the system must be static. Thus, it is necessary to perform a pre-processing of the data. For this, they will be used as the maximum static values of the peak values. The data used in the system has the behavior shown in Figure 4, where the first graph shows active data and the second, reactive data. The values show a collection made every half hour, for 72 hours, thus totaling 144 data for each bar.
	 
![Behavior of powers in 72 hours.](../figures/demand_level.png)
	 
For both tests, the result of the tension was the same, thus generating Figure 5. It shows the voltage level in p.u, in each of the bars. This unit, presented in the voltage, represents the division of the value of each of the bars, by the value of the base voltage. This is done to decrease the value to be calculated, changing, for example, the data 230 kV to 1 p.u, using the same basis of the tests.
	 
![Electricity distribution model with three bars.](../figures/voltage_completo.png)

	 
Figures 6 and 7 represent the power mismatches found in each iteration, for the Fast Newton and Fast Decoupled method, respectively. As explained earlier, the Newton Complete method tends to have fewer iterations, but it takes computational time, so for this test it was 0.0126 seconds, requiring 4 iterations. Fast Decoupled Newthon, it took an iteration more, but a computational time of 0.0069 seconds, about a 55\% reduction between the two methods.
	 

![Evolution of power mismatches for the Complete Newton method.](../figures/mismatches_completo.png)
        

![Evolution of power mismatches for the Fast Fast Decoupled method.](../figures/mismatches_fast.png)

# CONCLUSIONS

This work presented two methodologies for solving the load flow problem. As explained earlier, this problem is very important for the operation and planning of the electricity transmission system. Each of the methods presented has its advantages and disadvantages. Making the choice of which to use important, taking into account the accuracy of the results and the time it takes to calculate.

Performing the tests, it was possible to find that for the 14-bar system, there was a 55 \% decrease in the computational time required to find the system's operating point. For this system, the results found were the same. This would not happen for larger systems, since the computational differences and errors due to the adopted approaches would be more discrepant.


# REFERENCES

[1]  R. A. Jabr, “Radial distribution load flow using conic program-ming,”IEEE Transactions on Power Systems,  vol.  21,  no.  3,pp. 1458–1459, 2006.

[2]  F. Milano, “Continuous newton’s method for power flow anal-ysis,”IEEE Transactions on Power Systems, vol. 24, no. 1, pp.50–57, 2009.

[3]  M. Tostado-Véliz, S. Kamel, and F. Jurado, “Promising frame-work  based  on  multistep  continuous  newton  scheme  for  de-veloping  robust  pf  methods,”IET Generation, TransmissionDistribution, vol. 14, no. 2, pp. 265–274, 2020.

[4]  L.  H.  Macedo,  J.  F.  Franco,  M.  J.  Rider,  and  R.  Romero,“Optimal operation of distribution networks considering energystorage  devices,”IEEE Transactions on Smart Grid,  vol.  6,no. 6, pp. 2825–2836, 2015.

[5]  R.   V.   Anguita,   “Final   project,”   2020.   [Online].   Available:https://github.com/ReneJunior/final-project

[6]  R.   D.   Christie,   “Power   systems   test   case   archive,”   2020.[Online]. Available: labs.ece.uw.edu/pstca/